This script demos the calling of consensus ChIP seq peaks obtained from mouse chondrocyte tissue using consensusSeekeR
(https://www.bioconductor.org/packages/release/bioc/html/consensusSeekeR.html)

In [ ]:
##loading required packages
library(readr)
library(GenomicRanges)
library(rtracklayer)
library(dplyr)
library(consensusSeekeR)

In [ ]:
##setting working directory
setwd('D:/mouse_chondrocyte_chip_seq_macs2')

In [ ]:
dir <- file.path("D:", "mouse_chondrocyte_chip_seq_macs2")

In [ ]:
##importing a list csv file containing a list of narrowPeak files generated 
####from ChIP seq peak calling with MACS2
chondrocyte_narrowPeak_files <- read.csv("chondrocyte_narrowPeak_files.csv", header = FALSE)
##assign column name(s) to chondrocyte_narrowPeak_files
colnames(chondrocyte_narrowPeak_files) <- "chondrocyte_narrowPeak_filenames"

In [ ]:
chondrocyte_narrowPeak_files

In [ ]:
##importing the narrowPeak files obtained from peak calling with MACS2
chondrocyte1_L001_narrowPeak <- readNarrowPeakFile(paste0(dir, "/", chondrocyte_narrowPeak_files[1,1]), extractRegions = TRUE, extractPeaks = TRUE)
chondrocyte1_L002_narrowPeak <- readNarrowPeakFile(paste0(dir, "/", chondrocyte_narrowPeak_files[2,1]), extractRegions = TRUE, extractPeaks = TRUE)
chondrocyte2_L001_narrowPeak <- readNarrowPeakFile(paste0(dir, "/", chondrocyte_narrowPeak_files[3,1]), extractRegions = TRUE, extractPeaks = TRUE)
chondrocyte2_L002_narrowPeak <- readNarrowPeakFile(paste0(dir, "/", chondrocyte_narrowPeak_files[4,1]), extractRegions = TRUE, extractPeaks = TRUE)

In [ ]:
##taking a look at what a narrowPeak file looks like
##the first 6 columns of a narrowPeak file are essentially BED
##the last 4 columns of a narrowPeak file provides signalValue (overall enrichment of a region),
####statistical significance of the called peak (p and q values) and
####peak (the point source called for a peak)
chondrocyte1_L001_narrowPeak

In [ ]:
##converting the narrowPeak files into data frames
chondrocyte1_L001_narrowPeak_df <- data.frame(chondrocyte1_L001_narrowPeak[["narrowPeak"]])
chondrocyte1_L002_narrowPeak_df <- data.frame(chondrocyte1_L002_narrowPeak[["narrowPeak"]])
chondrocyte2_L001_narrowPeak_df <- data.frame(chondrocyte2_L001_narrowPeak[["narrowPeak"]])
chondrocyte2_L002_narrowPeak_df <- data.frame(chondrocyte2_L002_narrowPeak[["narrowPeak"]])

In [ ]:
##it is more easy to see the contents of a narrowPeak file when in data frame format
##here we use the head function to view the first 5 rows
head(chondrocyte1_L001_narrowPeak_df, n=5)

In [ ]:
##extracting peak locations from narrowPeak files and storing into data frames
chondrocyte1_L001_peak_df <- data.frame(chondrocyte1_L001_narrowPeak[["peak"]])
chondrocyte1_L002_peak_df <- data.frame(chondrocyte1_L002_narrowPeak[["peak"]])
chondrocyte2_L001_peak_df <- data.frame(chondrocyte2_L001_narrowPeak[["peak"]])
chondrocyte2_L002_peak_df <- data.frame(chondrocyte2_L002_narrowPeak[["peak"]])

In [ ]:
##the difference between chondrocyte1_L001_narrowPeak_df and chondrocyte1_L001_peak_df
####is that the peak width in chondrocyte1_L001_peak_df is 1
head(chondrocyte1_L001_peak_df, n=5)

In [ ]:
names(chondrocyte1_L001_narrowPeak[["narrowPeak"]]) <- rep("chondrocyte1_L001", length(chondrocyte1_L001_narrowPeak[["narrowPeak"]]))
names(chondrocyte1_L001_narrowPeak[["peak"]]) <- rep("chondrocyte1_L001", length(chondrocyte1_L001_narrowPeak[["peak"]]))
names(chondrocyte1_L002_narrowPeak[["narrowPeak"]]) <- rep("chondrocyte1_L002", length(chondrocyte1_L002_narrowPeak[["narrowPeak"]]))
names(chondrocyte1_L002_narrowPeak[["peak"]]) <- rep("chondrocyte1_L002", length(chondrocyte1_L002_narrowPeak[["peak"]]))
names(chondrocyte2_L001_narrowPeak[["narrowPeak"]]) <- rep("chondrocyte2_L001", length(chondrocyte2_L001_narrowPeak[["narrowPeak"]]))
names(chondrocyte2_L001_narrowPeak[["peak"]]) <- rep("chondrocyte2_L001", length(chondrocyte2_L001_narrowPeak[["peak"]]))
names(chondrocyte2_L002_narrowPeak[["narrowPeak"]]) <- rep("chondrocyte2_L002", length(chondrocyte2_L002_narrowPeak[["narrowPeak"]]))
names(chondrocyte2_L002_narrowPeak[["peak"]]) <- rep("chondrocyte2_L002", length(chondrocyte2_L002_narrowPeak[["peak"]]))

In [ ]:
##notice that now the sample names have been included
head(chondrocyte1_L001_narrowPeak[["narrowPeak"]], n=5)

In [ ]:
##sets reference genome to mm10, which was used for sequence read alignment
mm10info <- Seqinfo(genome="mm10")

In [ ]:
##the lines below will find consensus peaks in the replicates

##different values of extendingSize are tested to find the one which optimal
####The extendingSize indicates the size of padding on both sides of the position
####of the peaks median to create the consensus region. The minimum size of the
####consensus region is equal to twice the value of the extendingSize parameter.

##an importan parameter is minNbrExp, which indicate the minimum number of
####experiments in which at least one peak must be present for a potential consensus
####region (in this case, there are 4 replicates, so we can consider consensus
####if a peak shows up in 3 of the 4 replicates - thus minNbrExp=3)

In [ ]:
## consensus peaks - extendingSize = 1
chondrocyte_consensus_extend1 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 1, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 10
chondrocyte_consensus_extend10 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 10, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 50
chondrocyte_consensus_extend50 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 50, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 100
chondrocyte_consensus_extend100 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 100, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 300
chondrocyte_consensus_extend300 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 300, 
            expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 500
chondrocyte_consensus_extend500 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 500, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 575
chondrocyte_consensus_extend575 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 575, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 600
chondrocyte_consensus_extend600 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 600, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 601
chondrocyte_consensus_extend601 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 601, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 750
chondrocyte_consensus_extend750 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 750, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## consensus peaks - extendingSize = 1000
chondrocyte_consensus_extend1000 <- findConsensusPeakRegions(
  narrowPeaks = c(chondrocyte1_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte1_L002_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L001_narrowPeak[["narrowPeak"]], 
                  chondrocyte2_L002_narrowPeak[["narrowPeak"]]),
  peaks = c(chondrocyte1_L001_narrowPeak[["peak"]],
            chondrocyte1_L002_narrowPeak[["peak"]],
            chondrocyte2_L001_narrowPeak[["peak"]],
            chondrocyte2_L002_narrowPeak[["peak"]]), chrInfo = mm10info, extendingSize = 1000, 
  expandToFitPeakRegion = FALSE, shrinkToFitPeakRegion = FALSE, minNbrExp = 3, nbrThreads = 1)

In [ ]:
## creates a vector extend_size containing all the extendingSize tested
extend_size=c(1,10,50,100,300,500,575,600,601,750,1000)

In [ ]:
a <- length(chondrocyte_consensus_extend1[["consensusRanges"]])
b <- length(chondrocyte_consensus_extend10[["consensusRanges"]])
c <- length(chondrocyte_consensus_extend50[["consensusRanges"]])
d <- length(chondrocyte_consensus_extend100[["consensusRanges"]])
e <- length(chondrocyte_consensus_extend300[["consensusRanges"]])
f <- length(chondrocyte_consensus_extend500[["consensusRanges"]])
g <- length(chondrocyte_consensus_extend575[["consensusRanges"]])
h <- length(chondrocyte_consensus_extend600[["consensusRanges"]])
i <- length(chondrocyte_consensus_extend601[["consensusRanges"]])
j <- length(chondrocyte_consensus_extend750[["consensusRanges"]])
k <- length(chondrocyte_consensus_extend1000[["consensusRanges"]])

In [ ]:
number_of_chondrocyte_consensus_peaks <- c(a,b,c,d,e,f,g,h,i,j,k)

In [ ]:
number_of_chondrocyte_consensus_peaks_df <- data.frame(number_of_chondrocyte_consensus_peaks)

In [ ]:
colnames(number_of_chondrocyte_consensus_peaks_df) <- "number_of_chondrocyte_consensus_peaks"

In [ ]:
number_of_chondrocyte_consensus_peaks_df$extend_size <- extend_size

In [ ]:
plot(extend_size,number_of_chondrocyte_consensus_peaks)
axis(side=1, at=seq(0,1000,by=100))